In [1]:
import flowmap.formats.ugrid
from tvtk.api import tvtk
import geojson
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
from tvtk.common import configure_input, configure_source_data

%matplotlib inline

In [2]:
grid = flowmap.formats.ugrid.UGrid(path='/Users/baart_f/data/rijnland/LeerdamWest/leerdamwest_map_case06.nc', src_epsg=28992)
crs = geojson.crs.Named(properties={"name": "urn:ogc:def:crs:EPSG::{:d}".format(grid.src_epsg)})

In [3]:
ugrid = grid.ugrid

In [4]:
grid.to_polydata()

In [5]:
poly = grid.to_polydata()
cells = [
    list(poly.get_cell(idx).points)
    for idx
    in range(poly.number_of_cells)
]

In [6]:
multipolygon = geojson.MultiPolygon(coordinates=cells, crs=crs)

In [7]:
feature = geojson.Feature(id='grid', geometry=multipolygon)

In [8]:
with open('test.json', 'w') as f:
    geojson.dump(feature, f)

In [12]:
features = geojson.FeatureCollection(features=[
    geojson.Feature(geometry=geojson.Polygon(coordinates=cell, crs=crs))
    for cell 
    in cells
])

In [13]:
with open('test2.json', 'w') as f:
    geojson.dump(features, f)